# Antibody Validation with IgFold - FIXED VERSION

**Setup:** Enable GPU (Runtime → Change runtime type → GPU)


In [ ]:
# Cell 1: Check GPU
import torch

if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ No GPU! Enable: Runtime → Change runtime type → GPU")

In [ ]:
# Cell 2: Install (FIXED - PyTorch 1.13)
print("Installing IgFold with PyTorch 1.13...")
print("This may take 3-4 minutes...\n")

# Uninstall existing packages
!pip uninstall -y -q torch torchvision torchaudio transformers accelerate huggingface_hub

# Install PyTorch 1.13.1 (last stable 1.x version - avoids weights_only issues)
print("Installing PyTorch 1.13.1 (CUDA 11.7)...")
!pip install -q torch==1.13.1+cu117 --index-url https://download.pytorch.org/whl/cu117

# Install compatible dependencies
print("Installing dependencies...")
!pip install -q huggingface-hub==0.17.3
!pip install -q transformers==4.35.0
!pip install -q accelerate==0.20.3

# Install IgFold
print("Installing IgFold...")
!pip install -q igfold

print("\n✅ Installation complete!")
print("\n⚠️ RESTART: Runtime → Restart runtime")
print("Then re-run Cell 1, skip Cell 2, run Cell 3")

In [ ]:
# Cell 3: Test IgFold
from igfold import IgFoldRunner
import tempfile
import numpy as np

print("Initializing IgFold...")
igfold = IgFoldRunner()
print("✅ IgFold loaded!\n")

heavy = "EVQLVESGGGLVQPGGSLRLSCAASGFTISDYAIHWVRQAPGKGLEWVAGITPAGGYTAYADSVKGRFTISADTSKNTAYLQMNSLRAEDTAVYYCARFVFFLPYAMDYWGQGTLVTVSS"
light = "DIQMTQSPSSLSASVGDRVTITCRASQDVSTAVAWYQQKPGKAPKLLIYSASFLYSGVPSRFSGSGSGTDFTLTISSSLQPEDFATYCQQSYTTPPTFGQGTKVEIKR"

print(f"Testing with sample antibody:")
print(f"  Heavy: {len(heavy)} aa")
print(f"  Light: {len(light)} aa")
print("\nPredicting structure... (30-60 seconds)")

with tempfile.NamedTemporaryFile(mode='w', suffix='.pdb', delete=False) as tmp:
    pdb_file = tmp.name

sequences = {'H': heavy, 'L': light}
igfold.fold(pdb_file=pdb_file, sequences=sequences, do_refine=False, do_renum=False)

with open(pdb_file, 'r') as f:
    pdb_string = f.read()

plddt_scores = []
for line in pdb_string.split('\n'):
    if line.startswith('ATOM'):
        try:
            bfactor = float(line[60:66].strip())
            plddt_scores.append(bfactor)
        except:
            pass

plddt_scores = np.array(plddt_scores)
plddt_scores = np.clip(plddt_scores * 100, 0, 100)
mean_plddt = plddt_scores.mean()

print(f"\n✅ Structure predicted!")
print(f"   Mean pLDDT: {mean_plddt:.2f}")
print(f"   Min pLDDT:  {plddt_scores.min():.2f}")
print(f"   Max pLDDT:  {plddt_scores.max():.2f}")

if mean_plddt > 70:
    print("\n🎉 Good quality!")
elif mean_plddt > 50:
    print("\n✅ Fair quality")
else:
    print("\n⚠️ Lower quality")

with open('test_antibody.pdb', 'w') as f:
    f.write(pdb_string)
    
print("\nPDB saved as: test_antibody.pdb")

In [ ]:
# Cell 4: Download PDB
from google.colab import files
files.download('test_antibody.pdb')
print("✅ Downloaded!")